In [1]:
import torch
from datasets import Dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)


In [2]:

# + id="vIGZL2_exPHd"
model = "mlops-ft"

# + id="HQK9XMtNDwtL"
# Create a tokenizer instance from the Hugging Face model specified by 'pretrained_model'
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model,  # Load the tokenizer for the chosen model
)

# Add a special token '[PAD]' to the tokenizer, which can be used for padding sequences
tokenizer.add_special_tokens(special_tokens_dict={'pad_token': '[PAD]'})

# Set the padding side to "right", which means padding will be added to the right side of sequences
tokenizer.padding_side = "right"

# -

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [4]:
# Load the model for Causal Language Modeling (CausalLM) from the specified Hugging Face model
model = AutoModelForCausalLM.from_pretrained(
    model,  # Load the model specified by the 'model' variable
    load_in_4bit=True,  # Load the model using 4-bit quantization for memory efficiency
    torch_dtype=torch.float16,  # Set the data type to 16-bit float for faster computations
    device_map="cuda:0",  # Use the GPU device 'cuda:0' for model inference
    quantization_config = bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:

llm_pipeline = pipeline(
    model=model,  # Specify the model for text generation
    tokenizer=tokenizer,  # Specify the tokenizer for text processing
    task='text-generation',  # Define the task as text generation
    max_new_tokens=1024,  # Limit the maximum number of generated tokens to 256
    temperature=0.1,  # Set the temperature parameter for text generation
    top_k=10,  # Set the top-k parameter for text generation
    do_sample=True,  # Allow sampling from the model's output
    num_return_sequences=1,  # Return a single generated sequence
    eos_token_id=tokenizer.eos_token_id,  # Specify the end-of-sequence token ID
    pad_token_id=tokenizer.pad_token_id  # Specify the padding token ID
)



In [6]:

def get_prompt(question):
    prompt = f"""[INST]Act as a Multiple Answer Spans Healthcare Question Answering helpful assistant and answer the user's questions in details with reasoning. Do not give any false information. In case you don't have answer, specify why the question can't be answered.
    
### Question:
{question}

### Answer:
"""
    return prompt




In [29]:
result = llm_pipeline(get_prompt(question='What types of exercise are best for people with asthma?'),return_full_text=False)

print(result[0]['generated_text'])


Aerobic exercise, such as swimming or cycling, can help you breathe better and make you feel better overall. You may also want to try breathing exercises, such as pursed-lip breathing, to help you breathe more slowly and deeply. You can also use a breathing device called a peak flow meter to see how well you're breathing. If your asthma is well-controlled, you can do almost any type of exercise you want. But if your asthma is not well-controlled, you may need to avoid certain types of exercise. For example, if you have asthma that's triggered by cold air, you may want to avoid outdoor activities in the winter. If you have asthma that's triggered by dust, you may want to avoid activities that involve sweeping or vacuuming. If you have asthma that's triggered by strong smells, you may want to avoid activities that involve cleaning or cooking. If you have asthma that's triggered by exercise, you may want to avoid activities that involve running, jumping, or other high-intensity activities

In [26]:
# +
result = llm_pipeline(get_prompt(question='How is obsessive-compulsive disorder diagnosed?'),return_full_text=False)

print(result[0]['generated_text'])
# -


A doctor can diagnose OCD by asking about symptoms and ruling out other conditions. There are no lab tests for OCD.

### Reasoning:
The doctor will ask about symptoms and how they affect your life. He or she may ask about: Thoughts or worries about harm coming to yourself or others Behaviors that you repeat to "make things right" or to prevent bad things from happening Cleanliness or orderliness Compulsions to check things, such as whether you turned off the stove or locked the door Avoidance of certain situations or objects, such as flying or being in a crowded place Emotional distress or anxiety about having the thoughts or doing the behaviors You may also be asked to fill out a questionnaire about your symptoms. There are no lab tests for OCD. The doctor may also ask about any other mental health conditions, substance abuse, and family history of mental illness. OCD is often treated with therapy and medication.

### Note:
The information provided here is for educational purposes onl

In [40]:
# +
result = llm_pipeline(get_prompt(question='When are you more likely to get a blood clot?'),return_full_text=False)
print(result[0]['generated_text'])
# -


You're more likely to get a blood clot if you have a family history of them. You're also more likely to get a blood clot if you're overweight or obese. You're more likely to get a blood clot if you smoke. You're more likely to get a blood clot if you have a heart problem or a problem with your lungs. You're more likely to get a blood clot if you have diabetes. You're more likely to get a blood clot if you have a problem with your kidneys. You're more likely to get a blood clot if you have a problem with your liver. You're more likely to get a blood clot if you have a problem with your pancreas. You're more likely to get a blood clot if you have a problem with your spleen. You're more likely to get a blood clot if you have a problem with your stomach. You're more likely to get a blood clot if you have a problem with your small intestine. You're more likely to get a blood clot if you have a problem with your large intestine. You're more likely to get a blood clot if you have a problem wi

In [17]:
# +
result = llm_pipeline(get_prompt(question='How should you lift objects to prevent back pain?'),return_full_text=False)
print(result[0]['generated_text'])
# -

Lifting objects improperly is a common cause of back pain. Here are some tips on how to lift objects safely and prevent injury. 1. Warm up before lifting: Before you start lifting, do some light exercise or stretching to get your muscles ready for the task. 2. Get a firm grip: Hold the object as close to your body as possible with your palms facing the object. 3. Keep your body aligned: Stand with your feet shoulder-width apart and bend your knees slightly. Keep your back straight and your head up. 4. Lift slowly: Slowly lift the object straight up and over your head, keeping your arms close to your body. 5. Hold the object securely: Keep the object as close to your body as possible and avoid twisting your spine. 6. Take breaks: Don't lift objects that are too heavy for you. If you do, take frequent breaks and rest. 7. Keep your work area clean: Keep your work area clean and free of clutter. This will make it easier to lift objects and reduce the risk of injury. 8. Use proper lifting e

In [13]:
# +
result = llm_pipeline(get_prompt(question='How can you be smart with antibiotics?'),return_full_text=False)

print(result[0]['generated_text'])
# -

You can be smart with antibiotics by taking them as directed, finishing the full course, and not taking them for viral infections.

### Reasoning:
Antibiotics are powerful drugs that can fight off bacterial infections. But they don't work on viruses. If you take antibiotics for a viral infection, you're wasting your money and the drug. And if you stop taking antibiotics before the full course is done, the bacteria can come back and multiply. That's why it's important to follow your doctor's directions and take the full course of antibiotics. You should also take them as directed. That means taking the pills at the right times and in the right way. If you don't take them as directed, the bacteria may not die. And if you take too many antibiotics, the bacteria can become resistant to the drugs. That means they can fight off the antibiotics and come back to cause infections. So be smart with antibiotics. Take them as directed. Finish the full course. And don't take them for viral infectio